# Ro5 and beyond Ro5 overview of compounds
This notebook calculates the distribution of compounds following the Ro5 and bRo5 properties.

# Import Modules

In [1]:
import json
from collections import defaultdict
import pandas as pd
from tqdm import tqdm

from rdkit.Chem import Draw
from rdkit import Chem

tqdm.pandas()

# Add path constants

In [2]:
DATA_DIR = "../data/processed"
MAPPING_DIR = "../data/mappings"
FIGURE_DIR = "../data/figures"

# Load data file

In [3]:
surechembl_df = pd.read_parquet(f"{DATA_DIR}/surechembl_unique_inchikey_dump.pq.gzip")
surechembl_df.head(2)

,SureChEMBL_ID,SMILES,InChIKey,PATENT_ID,PUBLICATION_DATE,Field,year
0,SCHEMBL4,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,FAKRSMQSSFJEIM-RQJHMYQMSA-N,EP-2842582-A2,2015-03-04,Description,2015
1757,SCHEMBL9,O=C(O)\C=C/C(=O)O.CCOC(=O)[C@H](CCC1=CC=CC=C1)...,OYFJQPXVCSSHAI-QFPUQLAESA-N,EP-2838373-A2,2015-02-25,Description,2015


# Filtering unique compounds per year

In [4]:
year_df = surechembl_df[["SMILES", "year"]].copy()
year_df.drop_duplicates(subset=["SMILES"], keep="first", inplace=True)

# Removal of salts from the compound list

In [5]:
compounds_with_salt = set()
compounds_without_salt = set()

for smile in tqdm(year_df["SMILES"].unique()):
    if "." in smile:
        compounds_with_salt.add(smile)
    else:
        compounds_without_salt.add(smile)

len(compounds_with_salt), len(compounds_without_salt)

100%|██████████| 10695979/10695979 [00:09<00:00, 1154810.55it/s]


(328882, 10367097)

# Load properties data dump

In [6]:
properties_dict = json.load(open(f"{MAPPING_DIR}/properties.json"))

In [7]:
properties_df = pd.DataFrame(properties_dict)
properties_df = properties_df.T
properties_df.reset_index(inplace=True)
properties_df.rename(columns={"index": "SMILES"}, inplace=True)
properties_df.head(2)

,SMILES,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral,n_ring,n_heteroatoms,so_atoms
0,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,217.077264,0.6279,3,2,3,57.61,0.777778,1,1,5,False
1,O=C(O)\C=C/C(=O)O.CCOC(=O)[C@H](CCC1=CC=CC=C1)...,492.210781,1.3164,7,4,11,170.54,0.458333,1,2,11,False


In [9]:
len(properties_df)

10695979

# Subset to non-salt compounds

In [10]:
property_annotated_smiles_df = properties_df[
    properties_df["SMILES"].isin(compounds_without_salt)
]
property_annotated_smiles_df.head(2)

,SMILES,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral,n_ring,n_heteroatoms,so_atoms
0,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,217.077264,0.6279,3,2,3,57.61,0.777778,1,1,5,False
3,CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N1C...,376.199822,1.6046,5,2,9,95.94,0.55,1,2,7,False


In [11]:
len(property_annotated_smiles_df)

10367097

In [12]:
property_combined_df = pd.merge(
    property_annotated_smiles_df, year_df, how="left", on=["SMILES"]
)
property_combined_df.head(2)

,SMILES,mw,logp,n_hba,n_hbd,rot_bonds,tpsa,fsp3,n_chiral,n_ring,n_heteroatoms,so_atoms,year
0,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,217.077264,0.6279,3,2,3,57.61,0.777778,1,1,5,False,2015
1,CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N1C...,376.199822,1.6046,5,2,9,95.94,0.55,1,2,7,False,2015


In [13]:
property_combined_df.to_parquet(
    f"{DATA_DIR}/property_annotated_smiles.pq.gzip", compression="gzip"
)

# Calculating Ro5 critera for compounds

In [14]:
def is_ro5_compliant(row):
    """Check if a molecule is Ro5 or bRo5 compliant."""
    if pd.isna(row.mw):
        return None

    if (
        row.mw <= 500
        and row.n_hba <= 10
        and row.n_hbd <= 5
        and row.logp <= 5
        and row.rot_bonds <= 10
        and row.tpsa <= 140
    ):
        return "Ro5"

    elif row.mw > 500:
        if (
            row.n_hba > 10
            or row.n_hbd > 5
            or row.logp > 7
            or row.rot_bonds > 20
            or row.tpsa > 200
        ):
            return "bRo5"
        else:
            return None

    return None

In [15]:
ro5_annotation = property_combined_df.progress_apply(is_ro5_compliant, axis=1)

100%|██████████| 10367097/10367097 [10:32<00:00, 16378.66it/s]


In [15]:
ro5_counts = ro5_annotation.value_counts().to_dict()
total = sum(ro5_counts.values())

for i in ro5_counts:
    print(f"{i}: {ro5_counts[i]} ({ro5_counts[i]/total*100:.2f}%)")

Ro5: 5677233 (75.94%)
bRo5: 1799063 (24.06%)


# Looking into the top 5 compounds within each category

The top five compounds were selected based on their number of patents

In [16]:
property_annotated_smiles_df["ro5"] = ro5_annotation

/var/folders/61/bn_r735d70z03hg56wh7hsg80000gn/T/ipykernel_28338/703675761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_annotated_smiles_df['ro5'] = ro5_annotation


In [17]:
patent_counter_dict = defaultdict(set)

for smile, patent_idx in tqdm(surechembl_df[["SMILES", "PATENT_ID"]].values):
    patent_counter_dict[smile].add(patent_idx)

patent_counter_dict = {k: len(v) for k, v in patent_counter_dict.items()}

100%|██████████| 21857225/21857225 [01:21<00:00, 267075.95it/s]


In [18]:
property_annotated_smiles_df["patent_count"] = property_annotated_smiles_df[
    "SMILES"
].map(patent_counter_dict)

/var/folders/61/bn_r735d70z03hg56wh7hsg80000gn/T/ipykernel_28338/4238674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_annotated_smiles_df['patent_count'] = property_annotated_smiles_df['SMILES'].map(patent_counter_dict)


In [19]:
surechembl_map = defaultdict(str)

for idx, smile in tqdm(surechembl_df[["SureChEMBL_ID", "SMILES"]].values):
    surechembl_map[smile] = idx

100%|██████████| 21857225/21857225 [00:47<00:00, 464397.73it/s]


In [20]:
subset_df = property_annotated_smiles_df[["SMILES", "ro5", "patent_count"]].copy()
subset_df["schembl"] = subset_df["SMILES"].map(surechembl_map)
subset_df.head()

,SMILES,ro5,patent_count,schembl
0,C[C@H](CS)C(=O)N1CCC[C@H]1C(O)=O,Ro5,8,SCHEMBL4
3,CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N1C...,Ro5,8,SCHEMBL18
4,CNCC(=O)N[C@@H](CCCNC(N)=N)C(=O)N[C@@H](C(C)C)...,bRo5,8,SCHEMBL23
5,CC1=CC=C(C=C1)S(O)(=O)=O,Ro5,8,SCHEMBL34
6,CCCCC1=NC(Cl)=C(CO)N1CC1=CC=C(C=C1)C1=C(C=CC=C...,Ro5,8,SCHEMBL60


In [21]:
subset_df["patent_count"].value_counts()

1    5384506
2    2395460
3    1193340
4     614292
5     314066
6     201698
8     141859
7     121876
Name: patent_count, dtype: int64

# Get the top compound hits

In [22]:
k = (
    subset_df[subset_df["ro5"] == "Ro5"]
    .sort_values(by="patent_count", ascending=False)
    .head(5)
)
mols = [Chem.MolFromSmiles(s) for s in k["SMILES"]]

for idx, mol in enumerate(mols):
    Draw.MolToFile(mol, f"{FIGURE_DIR}/ro5_mol{idx}.png", size=(600, 600))

In [23]:
m = (
    subset_df[subset_df["ro5"] == "bRo5"]
    .sort_values(by="patent_count", ascending=False)
    .head(5)
)
mols_2 = [Chem.MolFromSmiles(s) for s in m["SMILES"]]

for idx, mol in enumerate(mols_2):
    Draw.MolToFile(mol, f"{FIGURE_DIR}/bro5_mol{idx}.png", size=(600, 600))